<a href="https://colab.research.google.com/github/Midd98/tareas/blob/master/4.-algoritmo_strassen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiplicacion de matrices

Sean $A$ y $B$ dos matrices de tamaño $n\times n$. Ahora dividimos las matrices en submatrices $A_{i,j}$ y $B_{i,j}$ de tamaño $n/2 \times n/2$ 
$$	
A = \begin{pmatrix}
    A_{1,1} & A_{1,2} \\
    A_{2,1} & A_{2,2} \\
    \end{pmatrix}
$$

$$
B = \begin{pmatrix}
	B_{1,1} & B_{1,2} \\
	B_{2,1} & B_{2,2} \\
	\end{pmatrix}
$$
    
La multiplicaci\'on de las dos matrices puede ser resuelto como:

$$
A  \times B = \begin{pmatrix}
	C_{1,1} & C_{1,2} \\
	C_{2,1} & C_{2,2} \\
	\end{pmatrix}
$$

$$
A =\begin{pmatrix}
	A_{1,1} & A_{1,2} \\
	A_{2,1} & A_{2,2} \\
	\end{pmatrix} \times \begin{pmatrix}
	B_{1,1} & B_{1,2} \\
	B_{2,1} & B_{2,2} \\
\end{pmatrix}
$$

$$
C_{i,j}= \sum_{k=1}^n A_{i,k} \times B_{k,j}
$$


In [0]:
import numpy as np 


def default_matrix_multiplication(A, B):
    if get_matrix_dimensions(A) != get_matrix_dimensions(B):
        raise Exception(f'Both matrices are not the same dimension!')
    n = A.shape[0]
    C = [[0 for i in np.arange(n)] for j in np.arange(n)]
    for i in np.arange(n):
        for j in np.arange(n):
            for k in np.arange(n):
                C[i][j] += A[i,k] * B[k,j]
    return np.matrix(C)

# Algoritmo Strassen

El algoritmo Strassen es un metodo del tipo Divide y Vencer\'as para resolver el problema de la multiplicacion de matrices cuadradas. El algoritmo Strassen calcula las siguientes operaciones:

$$
p_1 = (a_{11}) \times (b_{12} - b_{22})\\
p_2 = (a_{11} + a_{12}) \times (b_{22})\\
p_3 = (a_{21} + a_{22}) \times (b_{11})\\         
p_4 = (a_{22}) \times (b_{21} - b_{11})\\         
p_5 = (a{11} + a_{22}) \times (b_{11} + b_{22})\\         
p_6 = (a_{12} - a_{22}) \times  (b_{21} + b_{22})\\   
p_7 = (a{11} - a_{21}) \times (b_{11} + b_{12})
$$

$$
C_{1,1}=p_5+p_4 - p_2 +p_6\\
C_{1,2}=p_1 + p_2\\
C_{2,1}=p_3 + p_4\\
C_{2,2}=p_1 + p_5 - p_3 +p_7
$$

El siguiente codigo implementa una version recursiva del algoritmo Strassen:

In [0]:
def split_matrix(A):
    if A.shape[0] % 2 != 0 or A.shape[1] % 2 != 0:
        raise Exception('Odd matrices are not supported!')
    matrix_length = A.shape[0]
    mid = matrix_length // 2
    a11 = A[:mid,:mid]
    a21 = A[mid:,:mid]
    a22 = A[mid:,mid:]
    a12 = A[:mid,mid:]
    
    return a11, a12, a21, a22

def get_matrix_dimensions(matrix):
    return matrix.shape

def strassen(A,B,base_case=2): 
    # Base case when size of matrices is 1x1 
    if get_matrix_dimensions(A) != get_matrix_dimensions(B):
        raise Exception(f'Both matrices are not the same dimension!')
    if get_matrix_dimensions(A) == (base_case, base_case):
        return default_matrix_multiplication(A, B)

    a11, a12, a21, a22 = split_matrix(A) 
    b11, b12, b21, b22 = split_matrix(B) 
  
    p1 = strassen(a11, b12 - b22)   
    p2 = strassen(a11 + a12, b22)         
    p3 = strassen(a21 + a22, b11)         
    p4 = strassen(a22, b21 - b11)         
    p5 = strassen(a11 + a22, b11 + b22)         
    p6 = strassen(a12 - a22, b21 + b22)   
    p7 = strassen(a11 - a21, b11 + b12)   
  
    c11 = p5 + p4 - p2 + p6   
    c12 = p1 + p2            
    c21 = p3 + p4             
    c22 = p1 + p5 - p3 - p7   
  
    C = np.vstack((np.hstack((c11, c12)), np.hstack((c21, c22))))  
    return C 

Los resultados de la implementacion iterativa y la version recursiva de los algoritmos de multiplicacion de matrices pueden ser comparados con los resutados de Numpy.

In [0]:
def compare_results(C,A,B,decimals=2):
    return np.array_equal(np.around(np.dot(A,B), decimals=decimals),
                          np.around(C,decimals=decimals))

Versión recursiva del mecanismo iterativo

In [0]:
def matrix_multiplication_recursivo(A, B, base_case = 2):
    if get_matrix_dimensions(A) != get_matrix_dimensions(B): # No es matriz cuadrada
        raise Exception(f'Both matrices are not the same dimension!')
    elif get_matrix_dimensions(A) == (base_case, base_case): # Nos encontramos en caso base, pasamos a iterativo
      return default_matrix_multiplication(A,B)
    else: # Trabajar de forma recursiva
      # Separar/obtener las submatrices de N = 2
      a11, a12, a21, a22 = split_matrix(A)
      b11, b12, b21, b22 = split_matrix(B)
      # Obtener los valores de la multiplicacion de los subindices de las matricez.
      c11 = matrix_multiplication_recursivo(a11,b11) + matrix_multiplication_recursivo(a12, b21)
      c12 = matrix_multiplication_recursivo(a11,b12) + matrix_multiplication_recursivo(a12, b22)
      c21 = matrix_multiplication_recursivo(a21,b11) + matrix_multiplication_recursivo(a22, b21)
      c22 = matrix_multiplication_recursivo(a21,b12) + matrix_multiplication_recursivo(a22, b22)
      #Formar el resultado de la multiplicación
      C = np.vstack((np.hstack((c11,c12)), np.hstack((c21,c22))))
      return C

In [27]:
n = 64
A = np.matrix(np.random.uniform(0.0, 1.0, n*n).reshape(n,n))
B = np.matrix(np.random.uniform(0.0, 1.0, n*n).reshape(n,n))

C=default_matrix_multiplication(A,B)
print('Resultado correcto iterativo : {0}'.format(compare_results(C,A,B,1)))

C2=strassen(A,B)
print('Resultado correcto strassen : {0}'.format(compare_results(C2,A,B,1)))

C3=matrix_multiplication_recursivo(A,B)

print('Resultado correcto multiplicación recursiva : {0}'.format(compare_results(C3,A,B,1)))

Resultado correcto iterativo : True
Resultado correcto strassen : True
Resultado correcto multiplicación recursiva : True


Se prueban los tiempos de ejecución anteriores

In [34]:
import time
import pandas
import IPython


# Crear matriz cuadrada de dimension n

dimensiones = [4,32, 64, 256, 512]
casos_base = [2,4,8,32]
for n in dimensiones:
  if(n != dimensiones[0]): print("#"*50)
  A = np.matrix(np.random.uniform(0.0, 1.0, n*n).reshape(n,n))
  B = np.matrix(np.random.uniform(0.0, 1.0, n*n).reshape(n,n))

  print("Matriz ("+str(n)+"x"+str(n)+")")
  table_data = {}
  for base_case in casos_base:
    t = time.clock()
    C = default_matrix_multiplication(A,B)
    t1 = time.clock() - t

    t = time.clock()
    C = strassen(A,B,base_case)
    t2 = time.clock() - t

    t = time.clock()
    C = matrix_multiplication_recursivo(A,B,base_case)
    t3 = time.clock() - t
    table_data.update({"Caso base "+str(base_case): [t1,t2,t3] })
  table_frame = pandas.DataFrame(table_data, index = ["Iterativo","Strassen","Recursivo"])
  IPython.display.display(table_frame)



Matriz (4x4)


,Caso base 2,Caso base 4,Caso base 8,Caso base 32
Iterativo,0.000803,0.000843,0.000368,0.000390
Strassen,0.001052,0.000931,0.000908,0.000864
Recursivo,0.000855,0.000362,0.000865,0.000866


##################################################
Matriz (32x32)


,Caso base 2,Caso base 4,Caso base 8,Caso base 32
Iterativo,0.073966,0.068798,0.077570,0.068751
Strassen,0.166505,0.156118,0.157048,0.069861
Recursivo,0.217940,0.221965,0.217305,0.068960


##################################################
Matriz (64x64)


,Caso base 2,Caso base 4,Caso base 8,Caso base 32
Iterativo,0.676840,0.541093,0.577358,0.716712
Strassen,1.133902,1.110406,1.115001,1.123352
Recursivo,1.754798,1.753009,1.747141,1.747420


##################################################
Matriz (256x256)


,Caso base 2,Caso base 4,Caso base 8,Caso base 32
Iterativo,34.807965,33.205263,33.076727,32.424100
Strassen,53.526329,51.667024,49.730474,49.247916
Recursivo,108.219652,104.773918,102.176214,100.426545


##################################################
Matriz (512x512)


,Caso base 2,Caso base 4,Caso base 8,Caso base 32
Iterativo,262.150762,252.016501,258.752948,284.806720
Strassen,331.645727,339.178402,340.081970,377.633817
Recursivo,771.194786,791.789525,806.485463,831.324782
